In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import scipy

from loading_data import get_data
from utils_maze import get_trials

In [ ]:
thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots")

In [ ]:
import info.r063d3 as info
# import info.r067d1 as info

In [ ]:
events, position, _, _, _ = get_data(info)

In [ ]:
spd = position.speed(t_smooth=None)

In [ ]:
plt.figure()
plt.hist(spd.data, bins=[0, 2, 4, 6, 8, 10, 15, 20, 40, 60, 80, 200])
print(np.mean(spd.data))

In [ ]:
huh = nept.run_threshold(position, thresh=6., t_smooth=0.5)
runs = huh.merge(gap=0.5)

In [ ]:
s = 16100
e = 16300
fig, ax = plt.subplots()
ax.plot(position.time, position.y, ".", ms=3)
for start, stop in zip(runs.starts, runs.stops):
    ax.fill_between([start, stop], np.max(position.y), color="k", alpha=0.2)
for start, stop in zip(huh.starts, huh.stops):
    ax.fill_between([start, stop], np.max(position.y)/2, color="c", alpha=0.3)
plt.xlim(s, e)
plt.show()

In [ ]:
huh.n_epochs, runs.n_epochs

In [ ]:
speed = position.speed(t_smooth=5.)

In [ ]:
plt.plot(speed.time, speed.data)
plt.axhline(4, color="k")
plt.axhline(10, color="r")
plt.show()